# 🤖 Nong-View AI 농업영상분석 플랫폼 - 통합 개발 노트북 v1.0

**프로젝트 현황**: 85% 완성 → 100% MVP 목표  
**핵심 블로커**: 데이터베이스 통합 (0% → 100%)  
**개발 날짜**: 2025-10-27  
**버전**: v1.0

## 📋 개발 진행 상황
- ✅ POD 모듈 (100% 완성)
- ✅ API 구조 (40% 완성)
- 🔄 데이터베이스 통합 (진행 중)
- ⏳ API-POD 연결
- ⏳ 테스트 및 배포


## 🔧 1. 환경 설정 및 의존성 설치

In [ ]:
# 필수 패키지 설치
!pip install fastapi uvicorn sqlalchemy alembic psycopg2-binary redis
!pip install pydantic[email] python-multipart
!pip install rasterio geopandas shapely fiona
!pip install ultralytics torch torchvision
!pip install pillow opencv-python numpy pandas
!pip install python-jose[cryptography] passlib[bcrypt]
!pip install pytest pytest-asyncio httpx

In [ ]:
# 기본 라이브러리 임포트
import os
import sys
import json
import logging
import asyncio
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any, Union
from pathlib import Path
from uuid import UUID, uuid4
import hashlib
import time

# 데이터 처리
import numpy as np
import pandas as pd
from PIL import Image
import cv2

# 지리정보 처리
import rasterio
import geopandas as gpd
from shapely.geometry import Point, Polygon, mapping
from rasterio.crs import CRS

# 웹 프레임워크
from fastapi import FastAPI, HTTPException, Depends, UploadFile, File, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, FileResponse
from pydantic import BaseModel, Field, validator

# 데이터베이스
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Float, Text, Boolean, ForeignKey, JSON
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session, relationship
from sqlalchemy.dialects.postgresql import UUID as PG_UUID

# AI/ML
from ultralytics import YOLO
import torch

# 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ 모든 라이브러리 임포트 완료")

## ⚙️ 2. 설정 및 구성

In [ ]:
# 설정 클래스 정의
class Settings:
    """애플리케이션 설정"""
    PROJECT_NAME: str = "Nong-View API"
    VERSION: str = "1.0.0"
    API_V1_STR: str = "/api/v1"
    ENVIRONMENT: str = "development"
    DEBUG: bool = True
    
    # 서버 설정
    HOST: str = "127.0.0.1"
    PORT: int = 8000
    
    # 데이터베이스 설정
    DATABASE_URL: str = "sqlite:///./nongview_test.db"
    
    # 파일 저장 설정
    UPLOAD_PATH: str = "./uploads"
    CROP_PATH: str = "./crops"
    EXPORT_PATH: str = "./exports"
    MAX_FILE_SIZE: int = 2 * 1024 * 1024 * 1024  # 2GB
    
    # AI 모델 설정
    MODEL_PATH: str = "./models"
    MAX_WORKERS: int = 4
    
    # 보안 설정
    SECRET_KEY: str = "dev-secret-key-change-in-production"
    ALGORITHM: str = "HS256"
    ACCESS_TOKEN_EXPIRE_MINUTES: int = 1440

settings = Settings()

# 디렉토리 생성
for path in [settings.UPLOAD_PATH, settings.CROP_PATH, settings.EXPORT_PATH, settings.MODEL_PATH]:
    Path(path).mkdir(parents=True, exist_ok=True)

print("✅ 설정 완료")
print(f"📁 업로드 경로: {settings.UPLOAD_PATH}")
print(f"🗄️ 데이터베이스: {settings.DATABASE_URL}")

## 🗄️ 3. 데이터베이스 모델 정의 (SQLAlchemy)

In [ ]:
# SQLAlchemy 설정
engine = create_engine(
    settings.DATABASE_URL,
    connect_args={"check_same_thread": False} if "sqlite" in settings.DATABASE_URL else {}
)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# 데이터베이스 모델 정의
class User(Base):
    """사용자 모델"""
    __tablename__ = "users"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    username = Column(String(50), unique=True, index=True, nullable=False)
    email = Column(String(100), unique=True, index=True, nullable=False)
    hashed_password = Column(String(100), nullable=False)
    is_active = Column(Boolean, default=True)
    is_superuser = Column(Boolean, default=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    # 관계
    images = relationship("Image", back_populates="owner")
    analyses = relationship("Analysis", back_populates="owner")


class Image(Base):
    """이미지 모델"""
    __tablename__ = "images"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    filename = Column(String(255), nullable=False)
    file_path = Column(String(500), nullable=False)
    file_size = Column(Integer, nullable=False)
    format = Column(String(20), nullable=False)
    status = Column(String(20), default="uploading")
    
    # 메타데이터
    description = Column(Text)
    region_name = Column(String(100))
    capture_date = Column(DateTime)
    drone_model = Column(String(100))
    camera_model = Column(String(100))
    altitude = Column(Float)
    overlap = Column(Float)
    tags = Column(JSON)  # List[str]
    
    # 지리 정보
    crs = Column(String(50))
    bounds = Column(JSON)  # {"minx": float, "miny": float, "maxx": float, "maxy": float}
    resolution = Column(Float)
    width = Column(Integer)
    height = Column(Integer)
    bands = Column(Integer)
    
    # 소유자 및 시간
    owner_id = Column(String, ForeignKey("users.id"), nullable=False)
    uploaded_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    # 관계
    owner = relationship("User", back_populates="images")
    analyses = relationship("Analysis", back_populates="image")
    crop_results = relationship("CropResult", back_populates="image")


class Analysis(Base):
    """분석 작업 모델"""
    __tablename__ = "analyses"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    name = Column(String(200), nullable=False)
    description = Column(Text)
    status = Column(String(20), default="pending")  # pending, running, completed, error
    
    # 분석 설정
    analysis_type = Column(String(50), nullable=False)  # crop, facility, landuse
    model_version = Column(String(50))
    confidence_threshold = Column(Float, default=0.5)
    config = Column(JSON)  # 분석 설정
    
    # 결과
    total_detections = Column(Integer, default=0)
    processing_time = Column(Float)
    error_message = Column(Text)
    result_data = Column(JSON)  # 분석 결과 요약
    
    # 관계
    image_id = Column(String, ForeignKey("images.id"), nullable=False)
    owner_id = Column(String, ForeignKey("users.id"), nullable=False)
    
    # 시간
    created_at = Column(DateTime, default=datetime.utcnow)
    started_at = Column(DateTime)
    completed_at = Column(DateTime)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    # 관계
    image = relationship("Image", back_populates="analyses")
    owner = relationship("User", back_populates="analyses")
    crop_results = relationship("CropResult", back_populates="analysis")
    exports = relationship("Export", back_populates="analysis")


class CropResult(Base):
    """크로핑 결과 모델"""
    __tablename__ = "crop_results"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    pnu = Column(String(19), nullable=False, index=True)  # 필지고유번호
    crop_path = Column(String(500), nullable=False)
    crop_size = Column(Integer)
    
    # 지오메트리 정보
    geometry_type = Column(String(50))
    area = Column(Float)
    perimeter = Column(Float)
    bounds = Column(JSON)
    
    # AI 분석 결과
    detected_class = Column(String(50))
    confidence = Column(Float)
    detection_count = Column(Integer, default=0)
    detection_data = Column(JSON)  # 상세 검출 결과
    
    # 관계
    image_id = Column(String, ForeignKey("images.id"), nullable=False)
    analysis_id = Column(String, ForeignKey("analyses.id"))
    
    # 시간
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    # 관계
    image = relationship("Image", back_populates="crop_results")
    analysis = relationship("Analysis", back_populates="crop_results")


class Export(Base):
    """내보내기 작업 모델"""
    __tablename__ = "exports"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    name = Column(String(200), nullable=False)
    export_type = Column(String(50), nullable=False)  # gpkg, geojson, csv, shapefile
    status = Column(String(20), default="pending")  # pending, processing, completed, error
    
    # 파일 정보
    file_path = Column(String(500))
    file_size = Column(Integer)
    download_count = Column(Integer, default=0)
    
    # 내보내기 설정
    include_geometry = Column(Boolean, default=True)
    include_metadata = Column(Boolean, default=True)
    filter_config = Column(JSON)  # 필터 설정
    
    # 오류 정보
    error_message = Column(Text)
    
    # 관계
    analysis_id = Column(String, ForeignKey("analyses.id"), nullable=False)
    
    # 시간
    created_at = Column(DateTime, default=datetime.utcnow)
    completed_at = Column(DateTime)
    expires_at = Column(DateTime)  # 다운로드 만료일
    
    # 관계
    analysis = relationship("Analysis", back_populates="exports")


# 데이터베이스 테이블 생성
Base.metadata.create_all(bind=engine)

print("✅ 데이터베이스 모델 생성 완료")
print(f"📊 생성된 테이블: {list(Base.metadata.tables.keys())}")

## 🔗 4. 데이터베이스 연결 관리

In [ ]:
# 데이터베이스 세션 의존성
def get_db():
    """데이터베이스 세션 생성"""
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# 테스트용 사용자 생성
def create_test_user(db: Session):
    """테스트용 사용자 생성"""
    existing_user = db.query(User).filter(User.username == "testuser").first()
    if not existing_user:
        test_user = User(
            username="testuser",
            email="test@example.com",
            hashed_password="hashed_password_here",
            is_active=True
        )
        db.add(test_user)
        db.commit()
        db.refresh(test_user)
        print(f"✅ 테스트 사용자 생성: {test_user.id}")
        return test_user
    else:
        print(f"ℹ️ 테스트 사용자 존재: {existing_user.id}")
        return existing_user

# 테스트 사용자 생성
with SessionLocal() as db:
    test_user = create_test_user(db)
    
print("✅ 데이터베이스 연결 관리 설정 완료")

## 🔌 5. POD 모듈 통합
### POD1: 데이터 수집 및 관리

In [ ]:
# POD1: 데이터 수집 및 관리 모듈
class DataRegistry:
    """중앙 데이터 레지스트리"""
    
    def __init__(self, storage_path: str):
        self.storage_path = Path(storage_path)
        self.storage_path.mkdir(parents=True, exist_ok=True)
    
    def extract_image_metadata(self, file_path: Path) -> dict:
        """이미지 메타데이터 추출"""
        try:
            with rasterio.open(file_path) as src:
                bounds = {
                    "minx": src.bounds.left,
                    "miny": src.bounds.bottom,
                    "maxx": src.bounds.right,
                    "maxy": src.bounds.top
                }
                
                # 해상도 계산
                res_x = (src.bounds.right - src.bounds.left) / src.width
                res_y = (src.bounds.top - src.bounds.bottom) / src.height
                resolution = (res_x + res_y) / 2
                
                metadata = {
                    "crs": str(src.crs) if src.crs else "EPSG:5186",
                    "resolution": resolution,
                    "bounds": bounds,
                    "width": src.width,
                    "height": src.height,
                    "bands": src.count,
                    "format": src.driver
                }
                
                return metadata
                
        except Exception as e:
            logger.error(f"메타데이터 추출 실패: {e}")
            raise
    
    def save_file(self, upload_file: UploadFile) -> tuple[str, int]:
        """파일 저장"""
        file_id = str(uuid4())
        file_extension = Path(upload_file.filename).suffix
        file_path = self.storage_path / f"{file_id}{file_extension}"
        
        # 파일 저장
        content = upload_file.file.read()
        with open(file_path, "wb") as f:
            f.write(content)
        
        return str(file_path), len(content)

# 전역 데이터 레지스트리 인스턴스
data_registry = DataRegistry(settings.UPLOAD_PATH)

print("✅ POD1 데이터 수집 모듈 통합 완료")

### POD2: 크로핑 엔진

In [ ]:
# POD2: 크로핑 엔진
class CroppingEngine:
    """ROI 추출 및 크로핑 엔진"""
    
    def __init__(self, output_dir: str):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
    
    def crop_by_shapes(self, image_path: str, shapefile_path: str) -> list:
        """Shapefile 기반 이미지 크로핑"""
        try:
            # Shapefile 로드
            gdf = gpd.read_file(shapefile_path)
            
            results = []
            
            with rasterio.open(image_path) as src:
                for idx, row in gdf.iterrows():
                    pnu = row.get('PNU', f'unknown_{idx}').zfill(19)
                    geometry = row.geometry
                    
                    # 크로핑 수행
                    try:
                        # 지오메트리로 마스킹
                        out_image, out_transform = rasterio.mask.mask(
                            src, [geometry], crop=True
                        )
                        
                        # 출력 파일 경로
                        crop_id = str(uuid4())
                        crop_filename = f"crop_{crop_id}_{pnu}.tif"
                        crop_path = self.output_dir / crop_filename
                        
                        # 크롭된 이미지 저장
                        out_meta = src.meta
                        out_meta.update({
                            "driver": "GTiff",
                            "height": out_image.shape[1],
                            "width": out_image.shape[2],
                            "transform": out_transform
                        })
                        
                        with rasterio.open(crop_path, "w", **out_meta) as dest:
                            dest.write(out_image)
                        
                        # 결과 정보
                        result = {
                            "crop_id": crop_id,
                            "pnu": pnu,
                            "crop_path": str(crop_path),
                            "crop_size": crop_path.stat().st_size,
                            "geometry_type": geometry.geom_type,
                            "area": geometry.area,
                            "perimeter": geometry.length,
                            "bounds": {
                                "minx": geometry.bounds[0],
                                "miny": geometry.bounds[1],
                                "maxx": geometry.bounds[2],
                                "maxy": geometry.bounds[3]
                            }
                        }
                        
                        results.append(result)
                        logger.info(f"크로핑 완료: {pnu}")
                        
                    except Exception as e:
                        logger.error(f"크로핑 실패 - PNU {pnu}: {e}")
                        continue
            
            return results
            
        except Exception as e:
            logger.error(f"크로핑 엔진 오류: {e}")
            raise

# 전역 크로핑 엔진 인스턴스
cropping_engine = CroppingEngine(settings.CROP_PATH)

print("✅ POD2 크로핑 엔진 통합 완료")

### POD4: AI 추론 엔진

In [ ]:
# POD4: AI 추론 엔진
class AIInferenceEngine:
    """YOLOv11 기반 AI 추론 엔진"""
    
    def __init__(self):
        self.models = {}
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        logger.info(f"AI 엔진 초기화 - 디바이스: {self.device}")
        
        # 클래스 매핑
        self.class_mappings = {
            'crop': {
                0: 'IRG',      # 이탈리안 라이그라스
                1: 'BARLEY',   # 보리
                2: 'WHEAT',    # 밀
                3: 'CORN_SILAGE',  # 옥수수사일리지
                4: 'HAY',      # 건초
                5: 'UNKNOWN'   # 미분류
            },
            'facility': {
                0: 'GREENHOUSE_SINGLE',  # 단동 비닐하우스
                1: 'GREENHOUSE_MULTI',   # 연동 비닐하우스
                2: 'STORAGE',            # 저장시설
                3: 'LIVESTOCK',          # 축사
                4: 'SILO',              # 사일로
                5: 'UNKNOWN'            # 미분류
            }
        }
    
    def load_model(self, model_type: str, model_path: str = None):
        """YOLO 모델 로드"""
        try:
            if model_path and Path(model_path).exists():
                model = YOLO(model_path)
            else:
                # 기본 YOLOv8 모델 사용 (테스트용)
                model = YOLO('yolov8n.pt')
                logger.warning(f"사용자 모델을 찾을 수 없어 기본 모델 사용: {model_type}")
            
            self.models[model_type] = model
            logger.info(f"모델 로드 완료: {model_type}")
            
        except Exception as e:
            logger.error(f"모델 로드 실패 - {model_type}: {e}")
            raise
    
    def predict(self, image_path: str, model_type: str, confidence: float = 0.5) -> dict:
        """이미지 예측 수행"""
        try:
            if model_type not in self.models:
                raise ValueError(f"모델이 로드되지 않음: {model_type}")
            
            model = self.models[model_type]
            
            # 예측 수행
            results = model(image_path, conf=confidence, device=self.device)
            
            detections = []
            
            for result in results:
                boxes = result.boxes
                if boxes is not None:
                    for box in boxes:
                        # 바운딩 박스 좌표
                        x1, y1, x2, y2 = box.xyxy[0].tolist()
                        confidence_score = box.conf[0].item()
                        class_id = int(box.cls[0].item())
                        
                        # 클래스 이름 매핑
                        class_name = self.class_mappings.get(model_type, {}).get(
                            class_id, f"class_{class_id}"
                        )
                        
                        detection = {
                            "class_id": class_id,
                            "class_name": class_name,
                            "confidence": confidence_score,
                            "bbox": [x1, y1, x2, y2],
                            "area": (x2 - x1) * (y2 - y1)
                        }
                        
                        detections.append(detection)
            
            return {
                "detections": detections,
                "detection_count": len(detections),
                "model_type": model_type,
                "confidence_threshold": confidence
            }
            
        except Exception as e:
            logger.error(f"예측 실패: {e}")
            raise
    
    def predict_batch(self, image_paths: list, model_type: str, confidence: float = 0.5) -> list:
        """배치 예측 수행"""
        results = []
        for image_path in image_paths:
            try:
                result = self.predict(image_path, model_type, confidence)
                result['image_path'] = image_path
                results.append(result)
            except Exception as e:
                logger.error(f"배치 예측 실패 - {image_path}: {e}")
                results.append({
                    'image_path': image_path,
                    'error': str(e)
                })
        
        return results

# 전역 AI 엔진 인스턴스
ai_engine = AIInferenceEngine()

# 기본 모델 로드 (테스트용)
try:
    ai_engine.load_model('crop')
    ai_engine.load_model('facility')
    print("✅ POD4 AI 추론 엔진 통합 완료")
except Exception as e:
    print(f"⚠️ AI 모델 로드 실패: {e}")
    print("기본 YOLO 모델을 다운로드합니다...")

## 📋 6. API 스키마 정의

In [ ]:
# API 응답 스키마
class APIResponse(BaseModel):
    """기본 API 응답"""
    success: bool = True
    message: str = ""
    data: Optional[Any] = None
    error: Optional[str] = None

# 이미지 관련 스키마
class ImageUploadRequest(BaseModel):
    """이미지 업로드 요청"""
    description: Optional[str] = None
    region_name: Optional[str] = None
    tags: List[str] = []

class ImageResponse(BaseModel):
    """이미지 응답"""
    id: str
    filename: str
    file_path: str
    file_size: int
    status: str
    format: str
    description: Optional[str] = None
    region_name: Optional[str] = None
    tags: List[str] = []
    metadata: Optional[dict] = None
    uploaded_at: datetime
    updated_at: datetime

# 분석 관련 스키마
class AnalysisRequest(BaseModel):
    """분석 요청"""
    name: str
    description: Optional[str] = None
    analysis_type: str  # crop, facility, landuse
    confidence_threshold: float = Field(0.5, ge=0.1, le=1.0)
    shapefile_path: Optional[str] = None  # 크로핑용 Shapefile

class AnalysisResponse(BaseModel):
    """분석 응답"""
    id: str
    name: str
    description: Optional[str] = None
    status: str
    analysis_type: str
    confidence_threshold: float
    total_detections: int = 0
    processing_time: Optional[float] = None
    error_message: Optional[str] = None
    created_at: datetime
    started_at: Optional[datetime] = None
    completed_at: Optional[datetime] = None

# 크롭 결과 스키마
class CropResultResponse(BaseModel):
    """크롭 결과 응답"""
    id: str
    pnu: str
    crop_path: str
    crop_size: int
    geometry_type: str
    area: float
    detected_class: Optional[str] = None
    confidence: Optional[float] = None
    detection_count: int = 0
    created_at: datetime

print("✅ API 스키마 정의 완료")

## 🚀 7. FastAPI 애플리케이션

In [ ]:
# FastAPI 애플리케이션 생성
app = FastAPI(
    title=settings.PROJECT_NAME,
    description="AI 기반 농업 영상 분석 플랫폼 API",
    version=settings.VERSION,
    docs_url="/api/docs",
    redoc_url="/api/redoc"
)

# CORS 미들웨어
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 헬스 체크
@app.get("/health")
async def health_check():
    """헬스 체크 엔드포인트"""
    return APIResponse(
        success=True,
        message="서비스가 정상 작동 중입니다",
        data={
            "timestamp": datetime.now().isoformat(),
            "version": settings.VERSION,
            "environment": settings.ENVIRONMENT
        }
    )

print("✅ FastAPI 애플리케이션 생성 완료")

## 📸 8. Images API 구현

In [ ]:
# Images API 엔드포인트
@app.post("/api/v1/images/upload", response_model=APIResponse)
async def upload_image(
    file: UploadFile = File(...),
    description: str = None,
    region_name: str = None,
    db: Session = Depends(get_db)
):
    """이미지 업로드"""
    try:
        # 파일 확장자 검증
        allowed_extensions = ['.tif', '.tiff', '.jp2']
        file_extension = Path(file.filename).suffix.lower()
        if file_extension not in allowed_extensions:
            raise HTTPException(
                status_code=400,
                detail=f"지원되지 않는 파일 형식: {file_extension}"
            )
        
        # 파일 저장
        file_path, file_size = data_registry.save_file(file)
        
        # 메타데이터 추출
        try:
            metadata = data_registry.extract_image_metadata(Path(file_path))
        except Exception as e:
            logger.warning(f"메타데이터 추출 실패: {e}")
            metadata = {}
        
        # 테스트 사용자 가져오기
        test_user = db.query(User).filter(User.username == "testuser").first()
        if not test_user:
            raise HTTPException(status_code=404, detail="테스트 사용자를 찾을 수 없습니다")
        
        # 데이터베이스에 이미지 정보 저장
        db_image = Image(
            filename=file.filename,
            file_path=file_path,
            file_size=file_size,
            format=file_extension.replace('.', ''),
            status="ready",
            description=description,
            region_name=region_name,
            owner_id=test_user.id,
            crs=metadata.get('crs'),
            bounds=metadata.get('bounds'),
            resolution=metadata.get('resolution'),
            width=metadata.get('width'),
            height=metadata.get('height'),
            bands=metadata.get('bands')
        )
        
        db.add(db_image)
        db.commit()
        db.refresh(db_image)
        
        # 응답 데이터 구성
        response_data = ImageResponse(
            id=db_image.id,
            filename=db_image.filename,
            file_path=db_image.file_path,
            file_size=db_image.file_size,
            status=db_image.status,
            format=db_image.format,
            description=db_image.description,
            region_name=db_image.region_name,
            tags=[],
            metadata=metadata,
            uploaded_at=db_image.uploaded_at,
            updated_at=db_image.updated_at
        )
        
        return APIResponse(
            success=True,
            message="이미지 업로드 완료",
            data=response_data.dict()
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"이미지 업로드 실패: {e}")
        raise HTTPException(status_code=500, detail=f"이미지 업로드 실패: {str(e)}")


@app.get("/api/v1/images", response_model=APIResponse)
async def list_images(db: Session = Depends(get_db)):
    """이미지 목록 조회"""
    try:
        images = db.query(Image).all()
        
        image_list = []
        for img in images:
            image_data = ImageResponse(
                id=img.id,
                filename=img.filename,
                file_path=img.file_path,
                file_size=img.file_size,
                status=img.status,
                format=img.format,
                description=img.description,
                region_name=img.region_name,
                tags=img.tags or [],
                uploaded_at=img.uploaded_at,
                updated_at=img.updated_at
            )
            image_list.append(image_data.dict())
        
        return APIResponse(
            success=True,
            message=f"{len(image_list)}개의 이미지를 찾았습니다",
            data={"images": image_list, "total": len(image_list)}
        )
        
    except Exception as e:
        logger.error(f"이미지 목록 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"이미지 목록 조회 실패: {str(e)}")


@app.get("/api/v1/images/{image_id}", response_model=APIResponse)
async def get_image(image_id: str, db: Session = Depends(get_db)):
    """이미지 상세 조회"""
    try:
        image = db.query(Image).filter(Image.id == image_id).first()
        if not image:
            raise HTTPException(status_code=404, detail="이미지를 찾을 수 없습니다")
        
        image_data = ImageResponse(
            id=image.id,
            filename=image.filename,
            file_path=image.file_path,
            file_size=image.file_size,
            status=image.status,
            format=image.format,
            description=image.description,
            region_name=image.region_name,
            tags=image.tags or [],
            metadata={
                "crs": image.crs,
                "bounds": image.bounds,
                "resolution": image.resolution,
                "width": image.width,
                "height": image.height,
                "bands": image.bands
            },
            uploaded_at=image.uploaded_at,
            updated_at=image.updated_at
        )
        
        return APIResponse(
            success=True,
            message="이미지 조회 완료",
            data=image_data.dict()
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"이미지 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"이미지 조회 실패: {str(e)}")

print("✅ Images API 구현 완료")

## 🔬 9. Analysis API 구현

In [ ]:
# Analysis API 엔드포인트
@app.post("/api/v1/analyses", response_model=APIResponse)
async def create_analysis(
    image_id: str,
    request: AnalysisRequest,
    background_tasks: BackgroundTasks,
    db: Session = Depends(get_db)
):
    """분석 작업 생성"""
    try:
        # 이미지 존재 확인
        image = db.query(Image).filter(Image.id == image_id).first()
        if not image:
            raise HTTPException(status_code=404, detail="이미지를 찾을 수 없습니다")
        
        # 테스트 사용자 가져오기
        test_user = db.query(User).filter(User.username == "testuser").first()
        if not test_user:
            raise HTTPException(status_code=404, detail="테스트 사용자를 찾을 수 없습니다")
        
        # 분석 작업 생성
        analysis = Analysis(
            name=request.name,
            description=request.description,
            analysis_type=request.analysis_type,
            confidence_threshold=request.confidence_threshold,
            image_id=image_id,
            owner_id=test_user.id,
            status="pending"
        )
        
        db.add(analysis)
        db.commit()
        db.refresh(analysis)
        
        # 백그라운드에서 분석 실행
        background_tasks.add_task(
            run_analysis_task,
            analysis.id,
            image.file_path,
            request.analysis_type,
            request.confidence_threshold,
            request.shapefile_path
        )
        
        response_data = AnalysisResponse(
            id=analysis.id,
            name=analysis.name,
            description=analysis.description,
            status=analysis.status,
            analysis_type=analysis.analysis_type,
            confidence_threshold=analysis.confidence_threshold,
            created_at=analysis.created_at
        )
        
        return APIResponse(
            success=True,
            message="분석 작업이 시작되었습니다",
            data=response_data.dict()
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"분석 작업 생성 실패: {e}")
        raise HTTPException(status_code=500, detail=f"분석 작업 생성 실패: {str(e)}")


def run_analysis_task(
    analysis_id: str,
    image_path: str,
    analysis_type: str,
    confidence_threshold: float,
    shapefile_path: str = None
):
    """백그라운드 분석 작업 실행"""
    start_time = time.time()
    
    with SessionLocal() as db:
        try:
            # 분석 상태 업데이트
            analysis = db.query(Analysis).filter(Analysis.id == analysis_id).first()
            if not analysis:
                logger.error(f"분석 작업을 찾을 수 없음: {analysis_id}")
                return
            
            analysis.status = "running"
            analysis.started_at = datetime.utcnow()
            db.commit()
            
            logger.info(f"분석 시작: {analysis_id} - {analysis_type}")
            
            # 1. 크로핑 (Shapefile이 제공된 경우)
            crop_results = []
            if shapefile_path and Path(shapefile_path).exists():
                logger.info("크로핑 수행 중...")
                crop_data = cropping_engine.crop_by_shapes(image_path, shapefile_path)
                
                # 크롭 결과를 데이터베이스에 저장
                for crop_info in crop_data:
                    crop_result = CropResult(
                        pnu=crop_info["pnu"],
                        crop_path=crop_info["crop_path"],
                        crop_size=crop_info["crop_size"],
                        geometry_type=crop_info["geometry_type"],
                        area=crop_info["area"],
                        perimeter=crop_info["perimeter"],
                        bounds=crop_info["bounds"],
                        image_id=analysis.image_id,
                        analysis_id=analysis_id
                    )
                    db.add(crop_result)
                    crop_results.append(crop_result)
                
                db.commit()
                logger.info(f"크로핑 완료: {len(crop_results)}개 결과")
            
            # 2. AI 추론 실행
            total_detections = 0
            
            if crop_results:
                # 크롭된 이미지들에 대해 AI 추론
                crop_paths = [cr.crop_path for cr in crop_results]
                prediction_results = ai_engine.predict_batch(
                    crop_paths, analysis_type, confidence_threshold
                )
                
                # 예측 결과를 크롭 결과에 업데이트
                for i, crop_result in enumerate(crop_results):
                    if i < len(prediction_results):
                        pred_result = prediction_results[i]
                        if 'error' not in pred_result:
                            detections = pred_result.get('detections', [])
                            if detections:
                                # 가장 높은 신뢰도의 클래스 선택
                                best_detection = max(detections, key=lambda x: x['confidence'])
                                crop_result.detected_class = best_detection['class_name']
                                crop_result.confidence = best_detection['confidence']
                            
                            crop_result.detection_count = len(detections)
                            crop_result.detection_data = pred_result
                            total_detections += len(detections)
                
                db.commit()
            else:
                # 전체 이미지에 대해 AI 추론
                prediction_result = ai_engine.predict(
                    image_path, analysis_type, confidence_threshold
                )
                total_detections = prediction_result.get('detection_count', 0)
                analysis.result_data = prediction_result
            
            # 분석 완료 처리
            processing_time = time.time() - start_time
            analysis.status = "completed"
            analysis.completed_at = datetime.utcnow()
            analysis.total_detections = total_detections
            analysis.processing_time = processing_time
            
            db.commit()
            
            logger.info(f"분석 완료: {analysis_id} - {total_detections}개 검출, {processing_time:.2f}초")
            
        except Exception as e:
            # 에러 처리
            logger.error(f"분석 실행 실패: {e}")
            analysis.status = "error"
            analysis.error_message = str(e)
            analysis.completed_at = datetime.utcnow()
            db.commit()


@app.get("/api/v1/analyses", response_model=APIResponse)
async def list_analyses(db: Session = Depends(get_db)):
    """분석 목록 조회"""
    try:
        analyses = db.query(Analysis).all()
        
        analysis_list = []
        for analysis in analyses:
            analysis_data = AnalysisResponse(
                id=analysis.id,
                name=analysis.name,
                description=analysis.description,
                status=analysis.status,
                analysis_type=analysis.analysis_type,
                confidence_threshold=analysis.confidence_threshold,
                total_detections=analysis.total_detections,
                processing_time=analysis.processing_time,
                error_message=analysis.error_message,
                created_at=analysis.created_at,
                started_at=analysis.started_at,
                completed_at=analysis.completed_at
            )
            analysis_list.append(analysis_data.dict())
        
        return APIResponse(
            success=True,
            message=f"{len(analysis_list)}개의 분석을 찾았습니다",
            data={"analyses": analysis_list, "total": len(analysis_list)}
        )
        
    except Exception as e:
        logger.error(f"분석 목록 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"분석 목록 조회 실패: {str(e)}")


@app.get("/api/v1/analyses/{analysis_id}", response_model=APIResponse)
async def get_analysis(analysis_id: str, db: Session = Depends(get_db)):
    """분석 상세 조회"""
    try:
        analysis = db.query(Analysis).filter(Analysis.id == analysis_id).first()
        if not analysis:
            raise HTTPException(status_code=404, detail="분석을 찾을 수 없습니다")
        
        # 크롭 결과도 함께 조회
        crop_results = db.query(CropResult).filter(CropResult.analysis_id == analysis_id).all()
        
        crop_result_list = []
        for crop in crop_results:
            crop_data = CropResultResponse(
                id=crop.id,
                pnu=crop.pnu,
                crop_path=crop.crop_path,
                crop_size=crop.crop_size,
                geometry_type=crop.geometry_type,
                area=crop.area,
                detected_class=crop.detected_class,
                confidence=crop.confidence,
                detection_count=crop.detection_count,
                created_at=crop.created_at
            )
            crop_result_list.append(crop_data.dict())
        
        analysis_data = AnalysisResponse(
            id=analysis.id,
            name=analysis.name,
            description=analysis.description,
            status=analysis.status,
            analysis_type=analysis.analysis_type,
            confidence_threshold=analysis.confidence_threshold,
            total_detections=analysis.total_detections,
            processing_time=analysis.processing_time,
            error_message=analysis.error_message,
            created_at=analysis.created_at,
            started_at=analysis.started_at,
            completed_at=analysis.completed_at
        )
        
        return APIResponse(
            success=True,
            message="분석 조회 완료",
            data={
                "analysis": analysis_data.dict(),
                "crop_results": crop_result_list,
                "result_data": analysis.result_data
            }
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"분석 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"분석 조회 실패: {str(e)}")

print("✅ Analysis API 구현 완료")

## 🖥️ 10. 서버 시작 및 테스트

In [ ]:
# 서버 시작 (비동기)
import threading
import uvicorn

def start_server():
    """서버 시작 함수"""
    uvicorn.run(
        app,
        host=settings.HOST,
        port=settings.PORT,
        log_level="info"
    )

# 백그라운드에서 서버 시작
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

print(f"🚀 서버가 시작되었습니다!")
print(f"📍 URL: http://{settings.HOST}:{settings.PORT}")
print(f"📖 API 문서: http://{settings.HOST}:{settings.PORT}/api/docs")
print(f"🔍 헬스 체크: http://{settings.HOST}:{settings.PORT}/health")

# 잠시 대기 후 헬스 체크
import time
time.sleep(3)

try:
    import requests
    response = requests.get(f"http://{settings.HOST}:{settings.PORT}/health")
    if response.status_code == 200:
        print("✅ 서버 헬스 체크 성공!")
        print(f"응답: {response.json()}")
    else:
        print(f"⚠️ 헬스 체크 실패: {response.status_code}")
except Exception as e:
    print(f"⚠️ 헬스 체크 중 오류: {e}")
    print("서버가 아직 시작 중일 수 있습니다.")

## 🧪 11. API 테스트

In [ ]:
# API 테스트 함수들
import requests
import json

BASE_URL = f"http://{settings.HOST}:{settings.PORT}/api/v1"

def test_health_check():
    """헬스 체크 테스트"""
    try:
        response = requests.get(f"http://{settings.HOST}:{settings.PORT}/health")
        print(f"📋 헬스 체크: {response.status_code}")
        if response.status_code == 200:
            print(json.dumps(response.json(), indent=2, ensure_ascii=False))
        return response.status_code == 200
    except Exception as e:
        print(f"❌ 헬스 체크 실패: {e}")
        return False

def test_image_list():
    """이미지 목록 조회 테스트"""
    try:
        response = requests.get(f"{BASE_URL}/images")
        print(f"📸 이미지 목록: {response.status_code}")
        if response.status_code == 200:
            data = response.json()
            print(f"응답: {json.dumps(data, indent=2, ensure_ascii=False)}")
        return response.status_code == 200
    except Exception as e:
        print(f"❌ 이미지 목록 조회 실패: {e}")
        return False

def test_analysis_list():
    """분석 목록 조회 테스트"""
    try:
        response = requests.get(f"{BASE_URL}/analyses")
        print(f"🔬 분석 목록: {response.status_code}")
        if response.status_code == 200:
            data = response.json()
            print(f"응답: {json.dumps(data, indent=2, ensure_ascii=False)}")
        return response.status_code == 200
    except Exception as e:
        print(f"❌ 분석 목록 조회 실패: {e}")
        return False

# 테스트 실행
print("🧪 API 테스트 시작")
print("=" * 50)

tests = [
    ("헬스 체크", test_health_check),
    ("이미지 목록", test_image_list),
    ("분석 목록", test_analysis_list)
]

results = []
for test_name, test_func in tests:
    print(f"\n🔍 {test_name} 테스트:")
    result = test_func()
    results.append((test_name, result))
    print(f"결과: {'✅ 성공' if result else '❌ 실패'}")

print("\n" + "=" * 50)
print("📊 테스트 결과 요약:")
for test_name, result in results:
    print(f"  {test_name}: {'✅' if result else '❌'}")

success_count = sum(1 for _, result in results if result)
print(f"\n총 {len(results)}개 테스트 중 {success_count}개 성공")

## 📊 12. 샘플 데이터 생성 및 테스트

In [ ]:
# 샘플 이미지 데이터 생성 (테스트용)
def create_sample_image():
    """테스트용 샘플 이미지 생성"""
    try:
        # 간단한 샘플 이미지 생성
        import numpy as np
        from PIL import Image as PILImage
        
        # 3채널 RGB 이미지 생성 (1000x1000)
        width, height = 1000, 1000
        
        # 농지를 모방한 패턴 생성
        img_array = np.zeros((height, width, 3), dtype=np.uint8)
        
        # 녹색 계열의 배경 (농지)
        img_array[:, :, 1] = 120  # 녹색 채널
        img_array[:, :, 0] = 60   # 빨간색 채널
        img_array[:, :, 2] = 40   # 파란색 채널
        
        # 몇 개의 사각형 패턴 추가 (농지 구획)
        for i in range(5):
            x1 = np.random.randint(0, width-200)
            y1 = np.random.randint(0, height-200)
            x2 = x1 + np.random.randint(100, 200)
            y2 = y1 + np.random.randint(100, 200)
            
            # 다른 색상의 작물 구역
            color = np.random.randint(80, 160, 3)
            img_array[y1:y2, x1:x2] = color
        
        # PIL 이미지로 변환
        pil_image = PILImage.fromarray(img_array)
        
        # 파일로 저장
        sample_path = Path(settings.UPLOAD_PATH) / "sample_farm_image.png"
        pil_image.save(sample_path)
        
        print(f"✅ 샘플 이미지 생성: {sample_path}")
        return str(sample_path)
        
    except Exception as e:
        print(f"❌ 샘플 이미지 생성 실패: {e}")
        return None

# 샘플 Shapefile 생성
def create_sample_shapefile():
    """테스트용 샘플 Shapefile 생성"""
    try:
        from shapely.geometry import Polygon
        import geopandas as gpd
        
        # 샘플 농지 경계 생성 (5개 구획)
        polygons = []
        pnus = []
        
        for i in range(5):
            # 임의의 사각형 경계 생성
            x_base = 200000 + i * 500  # UTM-K 좌표계 기준
            y_base = 400000 + i * 300
            
            polygon = Polygon([
                (x_base, y_base),
                (x_base + 400, y_base),
                (x_base + 400, y_base + 300),
                (x_base, y_base + 300),
                (x_base, y_base)
            ])
            
            polygons.append(polygon)
            pnus.append(f"4717000000{i:03d}0000{i:03d}")
        
        # GeoDataFrame 생성
        gdf = gpd.GeoDataFrame({
            'PNU': pnus,
            'AREA': [p.area for p in polygons],
            'geometry': polygons
        })
        
        # CRS 설정 (UTM-K)
        gdf.crs = "EPSG:5186"
        
        # Shapefile 저장
        shapefile_path = Path(settings.UPLOAD_PATH) / "sample_parcels.shp"
        gdf.to_file(shapefile_path)
        
        print(f"✅ 샘플 Shapefile 생성: {shapefile_path}")
        return str(shapefile_path)
        
    except Exception as e:
        print(f"❌ 샘플 Shapefile 생성 실패: {e}")
        return None

# 샘플 데이터 생성
print("📊 샘플 데이터 생성 중...")
sample_image_path = create_sample_image()
sample_shapefile_path = create_sample_shapefile()

if sample_image_path and sample_shapefile_path:
    print("✅ 모든 샘플 데이터 생성 완료")
    print(f"🖼️ 샘플 이미지: {sample_image_path}")
    print(f"🗺️ 샘플 Shapefile: {sample_shapefile_path}")
else:
    print("⚠️ 일부 샘플 데이터 생성 실패")

## 📈 13. 개발 완료 상태 및 다음 단계

In [ ]:
# 개발 완료 상태 체크
def check_development_status():
    """개발 완료 상태 체크"""
    print("🔍 Nong-View 개발 상태 체크")
    print("=" * 60)
    
    # 데이터베이스 테이블 확인
    with SessionLocal() as db:
        try:
            user_count = db.query(User).count()
            image_count = db.query(Image).count()
            analysis_count = db.query(Analysis).count()
            crop_count = db.query(CropResult).count()
            
            print(f"📊 데이터베이스 상태:")
            print(f"  👥 사용자: {user_count}명")
            print(f"  🖼️ 이미지: {image_count}개")
            print(f"  🔬 분석: {analysis_count}개")
            print(f"  ✂️ 크롭 결과: {crop_count}개")
            
        except Exception as e:
            print(f"❌ 데이터베이스 조회 실패: {e}")
    
    # 디렉토리 확인
    directories = {
        "업로드": settings.UPLOAD_PATH,
        "크롭": settings.CROP_PATH,
        "내보내기": settings.EXPORT_PATH,
        "모델": settings.MODEL_PATH
    }
    
    print(f"\n📁 디렉토리 상태:")
    for name, path in directories.items():
        path_obj = Path(path)
        exists = path_obj.exists()
        file_count = len(list(path_obj.glob('*'))) if exists else 0
        print(f"  {name}: {'✅' if exists else '❌'} ({file_count}개 파일)")
    
    # AI 모델 상태 확인
    print(f"\n🤖 AI 모델 상태:")
    for model_type in ai_engine.models:
        print(f"  {model_type}: ✅ 로드됨")
    
    # 완성도 계산
    completed_features = [
        "데이터베이스 모델",
        "POD1 데이터 수집", 
        "POD2 크로핑",
        "POD4 AI 추론",
        "Images API",
        "Analysis API",
        "API 테스트"
    ]
    
    remaining_features = [
        "POD3 타일링",
        "POD5 병합", 
        "POD6 GPKG 내보내기",
        "Statistics API",
        "인증 시스템",
        "프로덕션 배포"
    ]
    
    total_features = len(completed_features) + len(remaining_features)
    completion_rate = (len(completed_features) / total_features) * 100
    
    print(f"\n📈 전체 완성도: {completion_rate:.1f}%")
    print(f"✅ 완성된 기능 ({len(completed_features)}개):")
    for feature in completed_features:
        print(f"  - {feature}")
    
    print(f"\n⏳ 남은 기능 ({len(remaining_features)}개):")
    for feature in remaining_features:
        print(f"  - {feature}")
    
    print(f"\n🎯 다음 개발 우선순위:")
    next_steps = [
        "1. POD3 타일링 시스템 통합",
        "2. POD5 결과 병합 엔진 통합", 
        "3. POD6 GPKG 내보내기 통합",
        "4. Statistics API 구현",
        "5. 종합 테스트 및 디버깅",
        "6. 프로덕션 배포 준비"
    ]
    
    for step in next_steps:
        print(f"  {step}")

# 상태 체크 실행
check_development_status()

print("\n" + "=" * 60)
print("🎉 Nong-View v1.0 통합 개발 노트북 완성!")
print("📝 이 노트북에서 구현된 내용:")
print("  - 완전한 데이터베이스 모델 (SQLAlchemy)")
print("  - POD 모듈 통합 (데이터 수집, 크로핑, AI 추론)")
print("  - RESTful API (이미지 관리, 분석 실행)")
print("  - 백그라운드 작업 처리")
print("  - API 테스트 및 샘플 데이터")
print("\n🚀 서버 URL: http://127.0.0.1:8000/api/docs")